In [21]:
import os
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pathlib import Path

tadpoleD1D2File = Path("../data/TADPOLE_D1_D2.csv")

class SimpleSVM:
    def __init__(self):
        self.diagnosis_model = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', svm.SVC(kernel="linear", probability=True)),
        ])
        self.adas_model = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', svm.SVR(kernel="linear")),
        ])
        self.ventricles_model = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', svm.SVR(kernel="linear")),
        ])
    
    def train_model(self, model, train_df, X_train, var_name):
        # remove rows with NaN future values
        Y_train_var = train_df[var_name]
        not_nans = np.logical_not(np.isnan(Y_train_var))
        X_train_var = X_train[not_nans]
        Y_train_var = Y_train_var[not_nans]
        
        model.fit(X_train_var, Y_train_var)
    
    def pre_process(self, train_df):
        train_df = train_df.copy()
        train_df.replace({'DXCHANGE': {4: 2, 5: 3, 6: 3, 7: 1, 8: 2, 9: 1}})
        train_df = train_df.rename(columns={"DXCHANGE": "Diagnosis"})

        # Sort the dataframe based on age for each subject
        train_df = train_df.sort_values(by=['RID', 'Years_bl'])
        
        train_df["Ventricles_ICV"] = train_df["Ventricles"].values / train_df["ICV_bl"].values
        
        # Select features
        train_df = train_df[
            ["RID", "Diagnosis", "ADAS13", "Ventricles_ICV"]
        ]

        # Force values to numeric
        train_df = train_df.astype("float64", errors='ignore')
        
        # Get future value from each row's next row
        for predictor in ["Diagnosis", "ADAS13", 'Ventricles_ICV']:
            train_df["Future_"+predictor] = np.append(train_df[predictor].values[1:], np.NaN)
        
        # Drop each last row per patient
        train_df = train_df.drop(train_df.groupby('RID').tail(1).index.values)
        return train_df
    
    def train(self, train_df: pd.DataFrame):
        train_df = self.pre_process(train_df)
        
        # Select columns for training
        X_train = train_df[["Diagnosis", "ADAS13", "Ventricles_ICV"]]
        
        # fill NaNs with mean
        X_train = X_train.fillna(X_train.mean())
        
        self.train_model(self.diagnosis_model,  train_df, X_train, "Future_Diagnosis")
        self.train_model(self.adas_model,       train_df, X_train, "Future_ADAS13")
        self.train_model(self.ventricles_model, train_df, X_train, "Future_Ventricles_ICV")

    def predict(predict_df: pd.DataFrame):
        pass


df = pd.read_csv(tadpoleD1D2File)

model = SimpleSVM()
model.train(df)

/home/tom/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,7

In [22]:
model.adas_model.predict([[1.0, 18.67, 0.059573]])

array([19.70281387])

In [23]:
model.pre_process(df)

,RID,Diagnosis,ADAS13,Ventricles_ICV,Future_Diagnosis,Future_ADAS13,Future_Ventricles_ICV
0,2.0,1.0,18.67,0.059573,1.0,19.67,NaN
5723,2.0,1.0,19.67,NaN,1.0,20.00,NaN
5724,2.0,1.0,20.00,NaN,1.0,23.00,NaN
5725,2.0,1.0,23.00,NaN,NaN,NaN,NaN
5726,2.0,NaN,NaN,NaN,1.0,21.00,NaN
5727,2.0,1.0,21.00,NaN,NaN,NaN,NaN
5728,2.0,NaN,NaN,NaN,4.0,14.00,NaN
5729,2.0,4.0,14.00,NaN,NaN,NaN,NaN
5730,2.0,NaN,NaN,NaN,1.0,18.00,NaN
5731,2.0,1.0,18.00,NaN,NaN,NaN,NaN
